## Point Cloud Data Exploration

##### Mike Pieschl

This module utilizes a 3D visualization of the radar data against a 3D mesh of the object.

In [5]:
import copy
import os
import numpy as np
import pandas as pd
import open3d as o3d
import tensorflow as tf
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from plotly.subplots import make_subplots

import importlib
import MeshSampler
import PointNet
import PointCloudSet
import mat_ops

importlib.reload(MeshSampler)
importlib.reload(PointNet)
importlib.reload(PointCloudSet)
importlib.reload(mat_ops)

<module 'mat_ops' from '/mnt/c/AFIT/2025 - Summer/CSCE823/3. Assigments/Project/mat_ops.py'>

In [6]:
AFTR_MAX_RANGE = 30
RADAR_MAX_RANGE = 20.82

SCALE_RANGE = RADAR_MAX_RANGE / AFTR_MAX_RANGE

#### Import and pre-process radar data

In [7]:
def structure_dataframes(df_list: list, class_list: list):

    assert len(df_list) == len(class_list)

    out_df = pd.DataFrame({}, columns = ['id', 'set', 'xg', 'yg', 'zg', 'cam_xg', 'cam_yg', 'cam_zg', 'roll', 'pitch', 'yaw', 'velocity', 'strength', 'label'])
    
    for i, df in enumerate(df_list):

        curr_set = 0

        for row in range(df.shape[0]):
            try:

                if(float(df.loc[row, 'range']) * SCALE_RANGE > 1):

                    # Convert from radar rpy to global xyz
                    pos = mat_ops.convert_radar_to_global(
                        np.array([float(df.loc[row, 'range']) * SCALE_RANGE, float(df.loc[row, 'azimuth']), float(df.loc[row, 'elevation'])]),
                        np.array([float(df.loc[row, 'xr']), float(df.loc[row, 'yr']), float(df.loc[row, 'zr'])]),
                        np.array([float(df.loc[row, 'rroll']), float(df.loc[row, 'rpitch']), float(df.loc[row, 'ryaw'])])
                        )
                    
                    new_row = pd.DataFrame({
                        'id': row,
                        'set': curr_set,
                        'xg': pos[0],
                        'yg': pos[1],
                        'zg': pos[2],
                        'cam_xg': float(df.loc[row, 'xr']),
                        'cam_yg': float(df.loc[row, 'yr']),
                        'cam_zg': float(df.loc[row, 'zr']),
                        'roll': float(df.loc[row, 'rroll']), 
                        'pitch': float(df.loc[row, 'rpitch']), 
                        'yaw': float(df.loc[row, 'ryaw']),
                        'velocity': float(df.loc[row, 'velocity']),
                        'strength': float(df.loc[row, 'strength']),
                        'label': class_list[i]
                    }, index = [0])
                    
                    out_df = pd.concat([out_df, new_row], ignore_index = True)

            except: 
                curr_set += 1

    return out_df

maverick_df = pd.read_csv('data/point_log_maverick.csv')
maverick_df.columns = ['range', 'azimuth', 'elevation', 'velocity', 'strength', 'xr', 'yr', 'zr', 'rroll', 'rpitch', 'ryaw']

z4_df = pd.read_csv('data/point_log_z4.csv')
z4_df.columns = ['range', 'azimuth', 'elevation', 'velocity', 'strength', 'xr', 'yr', 'zr', 'rroll', 'rpitch', 'ryaw']

mmwcas_df = structure_dataframes([maverick_df, z4_df], ['maverick', 'z4'])
mmwcas = mmwcas_df.set_index(['set', 'id'])
mmwcas_df.to_csv('data/mmwcas_filtered.csv')

/tmp/ipykernel_23776/3121600406.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out_df = pd.concat([out_df, new_row], ignore_index = True)
